# Notebook for Calculating 

Basic diagnostic on precipitation - enable comparison between two simulations

- Plot monthly mean precipitation and precipitation difference between simulation 1 & 2
- Compute seasonal cycle of precipitation at one location
- Compute seasonal cycle of precipitation over selected area

# Running the notebook
Execute the cells 1 by 1 by taping `ctrl + enter` or `shift + enter`

__Make sure to upload 2 file before continuing the rest of the notebook__

If you want to change the longitude and latitude bounds look in the data extraction section.

# Imports

In [ ]:
# Download new color palette
#Crameri : 
#palettable.scientific.sequential / palettable.scientific.diverging
#from palettable.scientific.sequential import Oslo_10
#cmap=Oslo_10.mpl_colormap
#https://jiffyclub.github.io/palettable/#documentation
!pip install palettable
from palettable.scientific.sequential import Davos_10_r, Oleron_15


from palettable.scientific.diverging import Vik_10


In [ ]:
# File upload widegt
from ipywidgets import FileUpload
from io import BytesIO

# Data manipulation and storage
import xarray as xr
import numpy as np

# Holoviews graphing
import hvplot.xarray
import holoviews as hv
from holoviews import opts

In [ ]:
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
import cartopy.feature as cf

gv.extension('bokeh', 'matplotlib')

# Data Upload

In [ ]:
# Create file upload
# To load several files press shift with selecting - 
#all the files have to be selected at the same time.

file_upload = FileUpload(multiple=True)
file_upload

In [ ]:
bytes_data = BytesIO(file_upload.data[0])
ds = xr.open_dataset(bytes_data)

bytes_data = BytesIO(file_upload.data[1])
ds1 = xr.open_dataset(bytes_data)

In [ ]:
#Scale precipitation to convert in mm/day
ds['precip'] = ds['precip']*86400
ds1['precip'] = ds1['precip']*86400

In [ ]:
ds[['precip', 'phis']].precip.hvplot(title="Dataset 1",
                                     width=300,
                                     height=300) + ds1[['precip', 'phis']].precip.hvplot(title="Dataset 2",
                                                                                                   width=300,
                                                                                                   height=300)

In [ ]:
# Compute elevation from phis parameter
ds['alti'] = ds['phis']/9.81
ds1['alti'] = ds1['phis']/9.81
ds.alti.hvplot()


# Plot Maps

Plot monthly mean precipitation for simulation 1 & 2

In [ ]:
# Plot monthly precipitations
topo = gv.Dataset(ds).to(gv.Image, ['lon','lat'],'alti')

opts.defaults(opts.Image(cmap=Davos_10_r.mpl_colormap,color_levels=10,clim=(0,20),frame_width=400,frame_height=200))

ensemble = gv.Dataset(ds).to(gv.Image, ['lon', 'lat'], 'precip')
ensemble1 = gv.Dataset(ds1).to(gv.Image, ['lon', 'lat'], 'precip')
(ensemble.opts(
    title="Precipitation (mm/day)",projection=crs.Robinson())* 
 gv.operation.contours(topo,levels=[10]).opts(
     show_legend=False,line_width=1.5) + 
 ensemble1.opts(
     colorbar=True,title="Precipitation (mm/day)",projection=crs.Robinson())*
 gv.operation.contours(topo,levels=[10]).opts(
     show_legend=False),line_width=1.5)


Plot monthly mean precipitation anomaly between simulation 1 & 2

In [ ]:
# Compute difference between two simulations & zoom over specific region
difference = (ds.precip - ds1.precip)

In [ ]:
# Plot differences
(gv.Dataset(difference).to(gv.Image, ['lon', 'lat'], 'precip').opts(
    width=600,height=300,
    cmap=Vik_10.mpl_colormap,color_levels=20,clim=(-15,15),
    colorbar=True,
    xlabel='Longitude',ylabel='Latitude',
    title='Precipitation anomaly (mm/day)',
    projection=crs.Robinson())*
 gv.operation.contours(topo,levels=[10]).opts(show_legend=False,cmap='gray_r',line_width=1.5))

# Plot seasonal cycles at one point

Here we first zoom on extended area just for esthetic issues

In [ ]:
# Select the desired zone
# /!\ Slice works based on the ordering in coordinates, here lat goes from +90 -> -90 and lon from -180 -> +180
ds_zone = ds.sel(lat=slice(40, -40), lon=slice(-180, 180))
ds1_zone = ds1.sel(lat=slice(40, -40), lon=slice(-180, 180))
topo_zoom = gv.Dataset(ds_zone).to(gv.Image, ['lon','lat'],'alti')

zoom = gv.Dataset(ds_zone).to(gv.Image, ['lon', 'lat'], 'precip')
zoom1 = gv.Dataset(ds1_zone).to(gv.Image, ['lon', 'lat'], 'precip')


Then we compute cycle at one location and plot

In [ ]:
# Select lon & lat for a poit where to compute seasonal cycle.
ilat=-5
ilon=-30

precip_curve = hv.Curve(zoom.select(lon=ilon,lat=ilat), kdims=['time_counter'])

# Plot 
(zoom * gv.operation.contours(topo_zoom,levels=[10])  * gv.Points([(ilon,ilat)]) + precip_curve).opts(
    opts.Image(title="Precipitation (mm/day)"),
    opts.Points(color='k',size=5),
    opts.Contours(show_legend=False,line_width=1.5,cmap='gray_r',color_levels=0),
    opts.Curve(frame_width=400,frame_height=200))

# Plot seasonal cycle over specific area

In [ ]:
ds_precip = ds[['precip', 'alti']]
ds1_precip = ds1[['precip', 'alti']]


In [ ]:
# Select the desired zone
ulat = 10
ulon = -10
blat = -1
blon = -20

# /!\ Slice works based on the ordering in coordinates, here lat goes from +90 -> -90 and lon from -180 -> +180
ds_precip_zone = ds_precip.sel(lat=slice(ulat, blat), lon=slice(blon, ulon))
ds1_precip_zone = ds1_precip.sel(lat=slice(0, -20), lon=slice(-65, 0))

In [ ]:
# Plot precip over selected area
(zoom * gv.operation.contours(topo_zoom,levels=[10]) * gv.Rectangles([(blon, blat,ulon,ulat)])).opts(
    opts.Image(title="Precipitation (mm/day)"),
    opts.Contours(show_legend=False,line_width=1.5,cmap='gray_r',color_levels=0),
    opts.Rectangles(color=None,line_width=3))



In [ ]:
 ds_precip_zone.precip.hvplot.image(label='precip').opts(cmap=Davos_10_r.mpl_colormap,color_levels=10,clim=(0,20)) 

In [ ]:
#Plot elevation
#ds_precip_zone.alti.hvplot.image(label='alti').opts(cmap=Oleron_15.mpl_colormap,color_levels=10,clim=(0,1000))

In [ ]:
# Extract values above a threshold
threshold = 1
upper = ds_precip_zone.where(ds_precip_zone.alti >= threshold)
upper1 = ds1_precip_zone.where(ds1_precip_zone.alti >= threshold)
# Extract values below threshold but above sea level
#lower = ds_precip_zone.where((ds_precip_zone.alti > 0) & (ds_precip_zone.alti < threshold))

In [ ]:
#Plot precip with threshold for elevation
#upper.hvplot.image().opts(cmap=Davos_10_r.mpl_colormap,color_levels=10,clim=(0,20))

# Calculate the mean value for each time step

In [ ]:
upper.precip.mean(dim=['lat', 'lon']).plot.line() + upper1.precip.mean(dim=['lat', 'lon']).plot.line()

In [ ]:
#lower.hvplot.image()

In [ ]:
#lower.precip.mean(dim=['lat', 'lon']).plot.line();

In [ ]:
#lower.precip.sum(dim=['lat', 'lon']).plot() + upper.precip.sum(dim=['lat', 'lon']).plot();

# Calculate the sum over all locations at each time step

In [ ]:
upper.precip.sum(dim=['lat', 'lon']).plot.line();

# Calculate the cumlative sum for each time step

In [ ]:
upper.precip.sum(dim=['lat', 'lon']).cumsum().plot.line();

In [ ]:
#lower.precip.sum(dim=['lat', 'lon']).cumsum().plot.line();